# Import libraries

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Defining list with file names

In [2]:
files = [
    "E8 - szkoły (aktualizacja 09.2021)",
    "E8 - szkoły (aktualizacja 09.2022)",
    "E8 - szkoły (aktualizacja 09.2023)"
]

# Combining columns with correlated exams

In [3]:
exams = [
    "język polski",
    "matematyka",
    "język angielski",
    "język francuski",
    "język hiszpański",
    "język niemiecki",
    "język rosyjski",
    "język włoski"
]

In [4]:
for file in files:
    with open(f"../data/eighth_grade_exam/{file} - raw.csv", "r", encoding="utf-8-sig") as f:
        columns = f.readline().rstrip("\n").split(";")

    i = 0
    while columns[i] != "liczba zdających":
        i += 1

    for exam in exams:
        columns[i] = f"{exam}_{columns[i]}"
        i += 1
        while columns[i] != "liczba zdających":
            columns[i] = f"{exam}_{columns[i]}"
            i += 1

            if i+1 == len(columns):
                break

    df = pd.read_csv(f"../data/eighth_grade_exam/{file} - raw.csv", sep=";")
    df.columns = columns
    df.to_csv(f"../data/eighth_grade_exam/{file}.csv", index=False, sep=";")

# Combining datasets

In [5]:
df1 = pd.read_csv(f"../data/eighth_grade_exam/{files[0]}.csv", sep=";")
df1['rok'] = 2021

df2 = pd.read_csv(f"../data/eighth_grade_exam/{files[1]}.csv", sep=";")
df2['rok'] = 2022

df3 = pd.read_csv(f"../data/eighth_grade_exam/{files[2]}.csv", sep=";")
df3['rok'] = 2023

df = pd.concat([df1, df2, df2])
df.reset_index(drop=True, inplace=True)

# Converting colum names

In [6]:
def convertColumns(columns: list):
    for i in range(len(columns)):
        # lowercase
        columns[i] = columns[i].lower()

        # strip percentage
        columns[i] = columns[i].rstrip(" (%)")

        # replace space
        columns[i] = columns[i].replace(" ", "_").replace("_-_", "_")

        # replace polish diacritical characters
        converter = {polish: normal for polish, normal in zip(list("ąęćżźśńłó"), list("aeczzsnlo"))}
        columns[i] = "".join([converter[char] if char in converter.keys() else char for char in list(columns[i])])

In [7]:
columns = df.columns.to_list()
convertColumns(columns)
df.columns = columns

In [8]:
df.rename(columns={"modalna": "jezyk_wloski_modalna"}, inplace=True)

# Convert columns data type

In [9]:
for col in df.columns.to_list():
    if "liczba_zdajacych" in col:
        df[col].fillna(-1, inplace=True)
        df[col] = df[col].astype("int64")

# Add primary key

In [15]:
df.reset_index(inplace=True)
df.rename(columns={"index": "row_id"}, inplace=True)

# Save

In [16]:
df.to_csv("../data/eighth_grade_exam/eighth_grade_exam_szkoly.csv", index=False, sep=";")